In [1]:
import warnings
import os, sys, platform
import glob
from astropy.units import UnitsWarning
from pathlib import Path

# Ignorar warnings comunes de librerías externas
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UnitsWarning)
print("🔇 Warnings silenciados: UserWarning, FutureWarning")
# Desactivar Rich para evitar problemas de compatibilidad
os.environ["RICH_NO_RICH"] = "1"

# Asegurar que el directorio 'src' esté en el path de Python
src_path = Path("src").resolve()
if src_path not in sys.path:
    sys.path.append(str(src_path))
from src.script_1_eb import main as run_script_1

# Detectar entorno local/AWS y ejecutar el script
try:
    import sagemaker
    is_sagemaker = True
except ImportError:
    is_sagemaker = False

#existing = len(glob.glob("/home/ec2-user/backup/data/raw/kepler/*.csv")) + len(glob.glob("/home/ec2-user/backup/data/raw/tess/*.csv"))
existing = len(glob.glob("/home/ec2-user/backup/data/raw/tess/*.csv"))
print(f"🗃️ Curvas ya existentes en disco: {existing}", flush=True)

# Ejecutar con la opción adecuada
if is_sagemaker:
    print("🔁 Ejecutando en SageMaker → catálogo completo")
    # Solo TESS, solo pendientes
    run_script_1(mission="TESS", only_pending=True, max_workers=12)
else:
    print("💻 Ejecutando en entorno local → catálogo de prueba")
    run_script_1(use_sample=True)


🔇 Warnings silenciados: UserWarning, FutureWarning
📁 Usando RAW_DIR: /home/ec2-user/backup/data/raw


[05/16/25 12:52:48] INFO     Found credentials from IAM Role:                                   ]8;id=186713;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=859809;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
🗃️ Curvas ya existentes en disco: 0
🔁 Ejecutando en SageMaker → catálogo completo
[⬇] Descargando catálogos completos de Kepler y TESS...
[⬇] Descargando catálogo Kepler EB...
[📂] Cargando catálogo Kepler EB desde copia local...
[⬇] Descargando catálogo TESS EB...
[⬇] Generando CSV de entrada para descarga de curvas...
📝 CSV generado con 0 estrellas → data/lists/eb_ids.csv
[⬇] Descargando curvas de luz...

🚀 Procesando misión: TESS (only_pending=True)
📝 CSV generado con 0 estrellas → data/lists/eb_ids_tess_pendientes.csv
[⬇] Descargando 0 curvas en paralelo con 12 hilos...


🚀 Descargando curvas: 0it [00:00, ?it/s]


	📦 0 objetos procesados hasta ahora...
[✓] Descarga finalizada: 0 objetos procesados.
[⭢] Leyendo y fusionando curvas descargadas...


[📁] Usando 16 lotes ya existentes en /home/ec2-user/temp_batches
[⏳] Uniendo todos los lotes en /home/ec2-user/backup/processed/dataset_eb_tess.parquet...


📚 Uniendo lotes: 100%|██████████| 16/16 [03:14<00:00, 12.13s/it]


[✅] Dataset parquet construido → /home/ec2-user/backup/processed/dataset_eb_tess.parquet
🧹 Carpeta temporal eliminada: /home/ec2-user/temp_batches
[⏱] Tiempo total: 196.25 segundos


### BACKUPS ANTES DE APAGAR INSTANCIA

In [ ]:
import shutil, os
from pathlib import Path

# Crear carpeta de backup dentro de SageMaker
backup_dir = Path.home() / "SageMaker" / "backup"
backup_dir.mkdir(exist_ok=True)
print(f"📁 Carpeta de backup creada en: {backup_dir}")

# 🔁 NUEVO: Copiar curvas desde la nueva ubicación temporal
#raw_dir_kepler = Path("/home/ec2-user/backup/raw/kepler")
raw_dir_tess = Path("/home/ec2-user/backup/raw/tess")
if raw_dir.exists():
    #shutil.make_archive(backup_dir / "curvas_raw_kepler", "zip", raw_dir_kepler)
    #print("✅ Copiado: curvas en backup/raw/kepler → curvas_raw_kepler.zip")
    shutil.make_archive(backup_dir / "curvas_raw_tess", "zip", raw_dir_tess)
    print("✅ Copiado: curvas en backup/raw/tess → curvas_raw_tess.zip")
else:
    print("⚠️ No se encontró backup/raw")

# Copiar dataset procesado
processed_dir = Path("SageMaker/astro_transformer/data/processed")
if processed_dir.exists():
    for file in processed_dir.glob("*.parquet"):
        shutil.copy(file, backup_dir)
        print(f"✅ Copiado: {file.name}")
else:
    print("⚠️ No se encontró data/processed")

# Copiar log de errores si existe
error_log = Path("SageMaker/astro_transformer/logs/fallos_descarga.csv")
if error_log.exists():
    shutil.copy(error_log, backup_dir)
    print("✅ Copiado: logs/fallos_descarga.csv")

# Copiar caché de lightkurve si existe
lk_cache = Path.home() / ".lightkurve" / "cache"
if lk_cache.exists():
    shutil.make_archive(backup_dir / "lightkurve_cache", "zip", lk_cache)
    print("✅ Copiado: ~/.lightkurve/cache → lightkurve_cache.zip")

print("🎉 Backup completado.")

In [ ]:
# Borrar cache de lightcurve
!rm -rf ~/.lightkurve/cache/*